In this notebook, dense array and scipy sparse matrices constructor, basic methods and conversion are compared in terms of running times. Conclusions are:

* converting an ndarray to a sparse matrix (and back) is time consuming (160 ms and 20 ms), to be avoided
* csr-to-coo and coo-to-csr conversions are not time consuming (130-200 µs), but still may be avoided
* csr-to-csr conversion is not time consuming (25-75 µs), but still more than checking if an array is an instance of csr (130 ns)
* transpose, conjugate and adjoint of a sparse matrix are not time consuming (50-200 µs) but one may still save time by storing them

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
from numpy.linalg import norm
from numpy import trace
from numpy import identity
from numpy import argpartition
from numpy.linalg import multi_dot
from scipy.linalg import hadamard
from copy import deepcopy
from scipy.sparse import csr_matrix, coo_matrix
from numpy.random import permutation

import matplotlib.pyplot as plt

In [ ]:
def get_data(shape=(1024, 1024), n_nonzeros=None):
    size = np.product(shape)
    if n_nonzeros is None:
        n_nonzeros = int(np.sqrt(size)) * 2
    I_nonzero = permutation(size)[:n_nonzeros]
    A = np.zeros(shape, dtype=complex)
    A.flat[I_nonzero] = np.random.randn(n_nonzeros) + 1j * np.random.randn(n_nonzeros)
    return A

In [ ]:
A_ndarray = get_data(shape=(10, 3))
print(A_ndarray)

In [ ]:
A_shape = (4096, 4096)
A_ndarray = get_data(shape=A_shape)
A_csr = csr_matrix(A_ndarray)
A_coo = coo_matrix(A_ndarray)

In [ ]:
print('ndarray to csr:')
%timeit csr_matrix(A_ndarray)
print('ndarray to coo:')
%timeit coo_matrix(A_ndarray)
print('csr to ndarray:')
%timeit A_csr.toarray()
print('coo to ndarray:')
%timeit A_coo.toarray()

print('csr to csr:')
%timeit csr_matrix(A_csr)
print('coo to coo:')
%timeit coo_matrix(A_coo)
print('csr to coo:')
%timeit csr_matrix(A_coo)
%timeit A_coo.tocsr()
print('coo to csr:')
%timeit coo_matrix(A_csr)
%timeit A_csr.tocoo()


In [ ]:
print('transpose ndarray:')
%timeit A_ndarray.T
%timeit np.transpose(A_ndarray)
print('transpose csr:')
%timeit A_csr.transpose()
print('transpose coo:')
%timeit A_coo.transpose()
print('conjugate ndarray:')
%timeit np.conjugate(A_ndarray)
print('conjugate csr:')
%timeit A_csr.conjugate()
print('conjugate coo:')
%timeit A_coo.conjugate()
print('Adjoint ndarray:')
%timeit np.conjugate(A_ndarray.T)
print('Adjoint csr:')
%timeit A_csr.getH()
print('Adjoint coo:')
%timeit A_coo.getH()

In [ ]:
%timeit isinstance(A_ndarray, csr_matrix)
%timeit isinstance(A_csr, csr_matrix)